## This script is used to check the mocks we use

### Current version -- AbacusHF-v1: 

box_path = /global/cfs/projectdirs/desi/mocks/cai/abacus_HF/DR2_v1.0/\<realization\>/Boxes/\<tracer\>/abacus_HF_\<tracer\>_\<redshift\>_\<version\>_\<realization\>_clustering.dat.fit

cutsky_path = /global/cfs/projectdirs/desi/mocks/cai/abacus_HF/DR2_v1.0/\<realization\>/Cutsky/\<tracer\>/abacus_HF_\<tracer\>_\<redshift\>_\<version\>_\<realization\>_clustering.dat.fit


In [19]:
import sys
import fitsio
import numpy as np
import matplotlib.pyplot as plt
from astropy.table import Table,join,Column

sys.path.append('/global/homes/s/shengyu/desi_y3_redshift_errors/main/')
from helper import REDSHIFT_OVERALL, COLOR_OVERALL
from helper import REDSHIFT_VSMEAR, REDSHIFT_LSS_VSMEAR, REDSHIFT_CUBICBOX, EDGES, COLOR_TRACERS, GET_RECON_BIAS

ABACUSHF_DIR = f'/global/cfs/projectdirs/desi/mocks/cai/abacus_HF/DR2_v1.0'

In [20]:
data_fn = ABACUSHF_DIR+ '/AbacusSummit_base_c000_ph000/Boxes/LRG/abacus_HF_LRG_0p500_DR2_v1.0_AbacusSummit_base_c000_ph000_clustering.dat.fits'
data = Table.read(data_fn)

data

X,Y,Z,VX,VY,VZ,HALO_MASS,HALO_ID,IS_CENTRAL
float64,float64,float64,float64,float64,float64,float64,int64,bool
-998.6694946289062,-999.3896484375,-663.2906494140625,10.265740095216584,266.5780354885274,912.4722635791048,11614711933135.018,285001000,True
-998.5205688476562,-998.8516235351562,79.28369903564453,-296.7538491330803,-219.10250760012465,-487.40419042366636,6382080680890.969,917000000,True
-999.5436401367188,-998.4382934570312,435.6242370605469,456.4253304047588,128.4366984790704,-226.6451039622638,8438435163332.705,1220000000,True
-998.9406127929688,-997.9343872070312,-654.687255859375,131.00042639457968,-195.82872132666907,285.580229486173,16467708511697.516,10293000000,True
-998.6441040039062,-998.1624145507812,-561.6430053710938,400.57338530974215,234.34802893373353,354.3194946442444,26315010130692.867,10372000000,True
-999.5943603515625,-997.7403564453125,741.9681396484375,449.16264803281825,57.81801221476887,-308.5042222157557,6854514941472.455,11480000000,True
-999.2175903320312,-997.933837890625,741.7716674804688,365.9791048858073,250.22452165112568,-64.73018781029172,5308558186980.359,11480000001,True
-998.2859497070312,-996.9385986328125,-535.5625,-42.90200497335152,433.64797260336564,-233.24540389053539,32870035496260.99,20394000000,True
-998.6669921875,-996.2734985351562,-67.93316650390625,-270.757413955437,-119.22690816103739,-42.33676444491049,5066013812128.258,20792000000,True


10212172722033.732

In [ ]:
data_fn = ABACUSHF_DIR+ '/AbacusSummit_base_c000_ph000/CutSky/LRG/z0.500/cutsky_LRG_z0.500_AbacusSummit_base_c000_ph000.fits'
data = Table.read(data_fn)

data 

RA,DEC,Z_COSMO,STATUS,RAW_NZ,RAN_NUM_0_1,NZ,Z,HALO_ID,HALO_MASS,IS_CENTRAL
float32,float32,float32,int32,float32,float32,float32,float32,int64,float32,bool
17.72037,20.497307,0.010497979,42,0.0008973729,0.19810149,0.0,0.010076427,2200070010000000,20329437000000.0,True
18.358932,23.168785,0.010508889,42,0.0008973729,0.8007446,0.0,0.010558409,2200070010000001,10655080000000.0,True
21.42871,28.056387,0.011969762,42,0.0008973729,0.9682616,0.0,0.011915651,2400090013005000,6529716600000.0,True
24.864092,27.895302,0.013859776,34,0.0008973729,0.31342417,0.0,0.01309043,2700120016002000,11121187000000.0,True
3.5962653,24.446495,0.012670616,42,0.0008973729,0.6923226,0.0,0.012805834,2800010013001000,5574302400000.0,True
29.721039,4.029662,0.01364716,42,0.0008973729,0.89460665,0.0,0.014413564,2900160002004000,10104610000000.0,True
41.08026,2.3175607,0.016398711,43,0.0008973729,0.08504421,0.00042017066,0.016711902,3000270001000000,7008478000000.0,True
14.15197,19.966831,0.014396515,42,0.0008973729,0.039054785,0.0,0.01428256,3300070012005000,5643902000000.0,True
66.85678,-8.329948,0.008779634,42,0.0008973729,0.28777534,0.0,0.009790924,800191697001000,8316108400000.0,True
